# 쓰레드와 믹스인 개념 이해하기

* 난이도 : ★★★★☆☆☆☆☆☆


* 간단한 채팅 서버와 클라이언트를 구현하기 위한 사전 지식
* 쓰레드에 대한 기본 이해가 필요합니다.
* ThreadingMixIn 에 대한 이해가 필요합니다.
* 이전 강좌에서 다룬 클래스 상속에 대한 이해가 필요합니다.

## 쓰레드

프로그램은 프로세스와 쓰레드의 집합으로 구성되어있습니다. 프로세스는 운영체제 입장에선 프로그램의 인스턴스 객체로 쉽게 프로그램 그 자체를 프로세스라고 볼 수 있습니다. 쓰레드는 그 프로그램 내에서 작업을 하는 하나의 단위로 볼 수 있습니다. 다른 예로 국가를 운영체제 라고 보면 프로세스는 하나의 공장입니다. 그리고 그 공장에서 일하는 노동자를 쓰레드로 볼 수 있습니다. 


파이썬으로 작성한 프로그램은 기본적으로 하나의 메인 쓰레드에서 실행됩니다. 이 메인 쓰레드가 프로그램의 코드를 순차적으로 수행하게 되며 프로그램이 동작합니다.

In [ ]:
'''
쓰레드의 이해를 쉽게 돕기 위해 함수명을 한글로 작성해봤습니다.
'''
import time
def 주문받기():
    for i in range(5):
        print("주문받기 {}".format(i))
        time.sleep(1)

def 우편발송():
    for i in range(5):
        print("우편발송 {}".format(i))
        time.sleep(1)

# 쓰레드 = 일꾼
# 일꾼 한명은 2가지일을 동시에 할 수 없습니다.
# 따라서 주문받기 일이 끝나야만 우편발송 일을 하게 됩니다.
주문받기()
우편발송()

In [ ]:
import threading
import time

def 주문받기():
    for i in range(5):
        print("주문받기 {}".format(i))
        time.sleep(1)

def 우편발송():
    for i in range(5):
        print("우편발송 {}".format(i))
        time.sleep(1)

# 기존의 일꾼(쓰레드) 1명으로 처리하던 일을
# 일을 전담할 일꾼(쓰레드)을 2명 뽑아서 각각 일을 맡겼습니다.
# 여기서 기존의 일꾼(메인쓰레드)는 이 쓰레드를 관리하는 일을 하게 됩니다.
# th1 은 주문받기 일을 처리할 쓰레드고
# th2 는 우편발송 일을 처리할 쓰레드 입니다.
th1 = threading.Thread(target=주문받기)
th2 = threading.Thread(target=우편발송)

# 일 시작!!
th1.start()
th2.start()

### 데몬 쓰레드
* 기본적으로 생성된 쓰레드는 일이 끝나기 전까지는 쓰레드를 종료하지 않습니다.
* 만약 프로그램이 종료된다면 생성된 쓰레드가 작업중이라도 종료 시키기를 원한다면 daemon 속성을 주면 됩니다.
> th = threading.Thread(target=function)  
> **th.daemon = True**  
> th.start()  

## Mix-in 개념
* 믹스인은 단어 그대로 보면 무언가를 섞는다는 개념입니다.
* 공통적인 기능을 모아서 다른 클래스에 추가적인 속성이나 기능을 제공하는 개념입니다.
* 쉽게 A + B = C를 만드는데 C는 A와 B의 모든 기능을 발휘할 수 있습니다.

In [ ]:
# MixIn 클래스 작성
class CarMixIn:
    def ready(self):
        print("믹스인 레디")
        
    def start(self):
        # CarMixIn 클래스에선 Performance 클래스의 멤버 변수들을 사용할 수 있습니다.
        print("{} 가 {} 속도로 달립니다.".format(self.name, self.speed))

# Base 클래스 작성
class Performance():
    def __init__(self, name, speed):
        self.name = name
        self.speed = speed
        
        # 실제 이 함수는 CarMixIn 클래스에 있는 함수 입니다.
        # Performance 클래스가 MixIn으로 선언되지 않고
        # 그냥 생성되면 여기서 오류가 발생합니다.
        self.ready()

# CarMixIn 클래스와 Perfomance 클래스를 MixIn 하여 생성된 새로운 SuperCar 클래스 
class SuperCar(CarMixIn, Performance):
    def show_info(self):
        print("{} 는 {} 속도의 성능입니다.".format(self.name, self.speed))

# Performance 클래스의 생성자 __init__ 에서 name, speed를 필요로 하므로 
# SuperCar 생성시에 이 값을 인자로 넘겨줘야 합니다.
s = SuperCar("람보르", 300)

# SuperCar 로 생성된 s 는 CarMixIn 클래스와 Performance 클래스의 모든 함수를 사용할 수 있습니다.
s.show_info()
s.start()

# Perfomance 클래스 생성자의 self.ready() 가 오류 납니다.
p = Performance("포니", 100)

### 오버라이딩

* 오버라이딩은 클래스를 상속받아 새롭게 작성할때 기존의 부모가 되는 클래스의 함수를 무시하고 새로운 기능으로 함수를 작성하는 개념입니다.
* 만약 부모의 원래 함수도 호출을 하고 새롭게 오버라이딩 된 함수도 호출하려면 super() 메서드를 사용합니다.

In [2]:
# MixIn 클래스 작성
class CarMixIn:
    def start(self):
        print("{} 가 {} 속도로 달립니다.".format(self.name, self.speed))

# Base 클래스 작성
class Performance():
    def __init__(self, name, speed):
        self.name = name
        self.speed = speed

# CarMixIn 클래스와 Perfomance 클래스를 MixIn 하여 생성된 새로운 SuperCar 클래스 
class SuperCar(CarMixIn, Performance):
    # start 함수는 CarMixIn에 선언되어있지만 여기서 새롭게 오버라이딩 했습니다.
    def start(self):
        # 여기서 만약 부모(CarMixIn) 의 start()를 호출하려면 super() 메서드를 사용합니다.
        super().start()
        print("{} 는 {} 속도의 성능입니다.".format(self.name, self.speed))
        

s = SuperCar("람보르", 300) 
# SuperCar 클래스의 start 함수는 새롭게 오버라이딩 된 함수가 호출됩니다.
s.start()

람보르 가 300 속도로 달립니다.
람보르 는 300 속도의 성능입니다.
